### -----맨 윗줄에 있는 코드는 오디오 파일에서 음역대 추출하는 코드라서 로컬에서만 사용됨-----

In [ ]:
import os
import librosa
import pandas as pd

# 오디오를 특정 시간 범위로 자르는 함수
def trim_audio(y, start_time=0, sec=4000):
    sr = 16000
    ny = y[start_time * sr:sr * (sec + start_time)]
    return ny

# 오디오 파일에서 피치 정보를 추출하는 함수
def get_pitch(audio_path, hop_length=441, n_fft=882):
    y, sr = librosa.load(audio_path)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    df = pd.DataFrame(pitches.T, columns=[f'frame_{i}' for i in range(pitches.shape[0])])
    df['time_stamp'] = librosa.times_like(pitches, sr=sr)
    df = pd.melt(df, id_vars='time_stamp', value_name='pitch')
    df = df[df['pitch'] != 0]
    return df

# 경로 내의 모든 오디오 파일을 처리하고 결과를 저장하는 함수
def process_audio_folder(folder_path, output_base_folder, hop_length=441, n_fft=882):
    df_list = []  # 데이터프레임을 저장할 리스트

    # 경로 내의 모든 파일에 대해 반복 (파일 하나만 있어도 ok)
    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            audio_path = os.path.join(folder_path, filename)
            y, sr = librosa.load(audio_path)

            # 오디오를 특정 시간으로 자르기
            trimmed_y = trim_audio(y, start_time=0, sec=4000)
            # 피치 정보 추출
            df_pitch = get_pitch(audio_path, hop_length=hop_length, n_fft=n_fft)
            #피치 변화량 계산
            df_pitch['delta'] = df_pitch['pitch'].diff()
            df_pitch.loc[df_pitch['variable'].shift(1) != df_pitch['variable'], 'delta'] = None
            df_pitch['delta'] = df_pitch['delta'].round(3)
            df_pitch['delta'] = abs(df_pitch['delta'])
            df_pitch = df_pitch.dropna(subset=['delta'])
            # 파일명에서 첫번째로 나오는 _ 이전 값을 lecture_name으로 사용
            lecture_name_dynamic = filename.split('_')[0]

            # 선생님의 이름은 output_base_folder에서 추출
            teacher_name = output_base_folder.split('/')[-2]  # 예제 코드 기준으로 수정한 부분

            # 변수명 생성 및 동적 할당
            variable_name = f'{lecture_name_dynamic}_{teacher_name}_data'
            globals()[variable_name] = df_pitch

            # 'teacher', 'class_name', 'lecture_name' 컬럼 추가
            folder_elements = output_base_folder.split('/')
            class_name = folder_elements[-2].replace('[', '').replace(']', '').replace(' ', '_')
            df_pitch['teacher'] = teacher_name
            df_pitch['class_name'] = class_name
            df_pitch['lecture_name'] = lecture_name_dynamic
            df_list.append(df_pitch)
    # 모든 데이터프레임을 하나로 결합
    combined_df = pd.concat(df_list, ignore_index=True)
    # 결과 저장
    output_folder_combined = os.path.join(output_base_folder, f'{teacher_name}_combined_data.csv')
    combined_df.to_csv(output_folder_combined, index=False)

# 입력 폴더와 출력 폴더를 지정하고 함수 호출해서 사용
audio_folder = './EBSi/정유빈/[2024 수능특강] 정유빈의 확률과 통계/combined'
output_base_folder = './EBSi/정유빈/[2024 수능특강] 정유빈의 확률과 통계/combined'
process_audio_folder(audio_folder, output_base_folder, hop_length=441, n_fft=882)

In [ ]:
import pandas as pd

df1 = pd.read_csv('./yubin_24_pitch.csv')

In [ ]:
df2 = pd.read_csv('./gina_pitch.csv')
df2

,Unnamed: 0,time_stamp,teacher,class_name,lecture_name,interval,median_pitch,median_deviation
0,0,2.461315,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,0,2254.503000,-1393.329400
1,1,2.554195,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,1,1820.601200,-959.427600
2,2,9.450522,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,2,1650.150000,-788.976400
3,3,14.744671,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,3,798.236370,62.937230
4,4,21.060499,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,4,730.911105,130.262495
...,...,...,...,...,...,...,...,...
5095,5095,3228.130975,서지나,올림포스_확률과_통계,9강. 이항정리(3),95,895.024275,-1.641215
5096,5096,3261.149751,서지나,올림포스_확률과_통계,9강. 이항정리(3),96,914.148400,-20.765340
5097,5097,3296.699501,서지나,올림포스_확률과_통계,9강. 이항정리(3),97,865.465485,27.917575
5098,5098,3330.298776,서지나,올림포스_확률과_통계,9강. 이항정리(3),98,1350.528600,-457.145540


In [ ]:
import numpy as np
import pandas as pd

# 타임스탬프를 각 강의의 시작 지점을 1%, 마지막 지점을 100%로 백분율로 변환하는 함수
def convert_to_percentage_by_lecture(df):
    percentage_list = []

    for lecture_name, group in df.groupby('lecture_name'):
        start_timestamp = group['time_stamp'].min()
        end_timestamp = group['time_stamp'].max()

        group['Scaled_Percent'] = (group['time_stamp'] - start_timestamp) / (end_timestamp - start_timestamp) * 99 + 1
        percentage_list.append(group)

    return pd.concat(percentage_list)

# lecture_name이 같은 값에 해당하는 pitch의 중앙값을 계산하고 deviation 컬럼을 추가하는 함수
def calculate_deviation(df):
    deviation_list = []

    for (teacher, class_name, lecture_name), group in df.groupby(['teacher', 'class_name', 'lecture_name']):
        median_pitch = group['pitch'].median()

        # deviation 계산 및 데이터프레임에 추가
        group['deviation'] = median_pitch - group['pitch']
        deviation_list.append(group)

    return pd.concat(deviation_list)

# 주어진 간격에 따라 중간값의 피치와 deviation 중간값을 계산하는 함수
def calculate_interval_pitch_with_deviation(df, interval):
    df_with_deviation = calculate_deviation(df)
    interval_pitch_deviation = []

    for (teacher, class_name, lecture_name, interval), group in df_with_deviation.groupby(['teacher', 'class_name', 'lecture_name', 'interval']):
        median_pitch = group['pitch'].median()
        median_deviation = group['deviation'].median()

        interval_pitch_deviation.append({
            'time_stamp': group['time_stamp'].iloc[0],  # 첫 번째 타임스탬프 사용
            'teacher': teacher,
            'class_name': class_name,
            'lecture_name': lecture_name,
            'interval': interval,
            'median_pitch': median_pitch,
            'median_deviation': median_deviation
        })

    return pd.DataFrame(interval_pitch_deviation)

# 주어진 데이터프레임을 각 강의의 시작 지점을 1%, 마지막 지점을 100%로 백분율로 변환
df1 = convert_to_percentage_by_lecture(df1)
# lecture_name의 같은 값에 해당되는 강의에서 pitch의 중간값을 구하고 deviation 컬럼을 추가
df1_with_deviation = calculate_deviation(df1)
# interval 컬럼 추가 (간격을 1로 수정)
df1_with_deviation['interval'] = pd.cut(df1_with_deviation['Scaled_Percent'], bins=np.arange(0, 101, 1), labels=False)
# pitch 값과 deviation 값의 중간값을 계산한 데이터프레임 생성
df1_interval_pitch_deviation = calculate_interval_pitch_with_deviation(df1_with_deviation, 1)

df1_interval_pitch_deviation


,time_stamp,teacher,class_name,lecture_name,interval,median_pitch,median_deviation
0,0.046440,정유빈,2024_수능특강_정유빈의_확률과_통계,01강 원순열,0,1689.575000,-784.853780
1,1.718277,정유빈,2024_수능특강_정유빈의_확률과_통계,01강 원순열,1,1770.235000,-865.513780
2,19.597642,정유빈,2024_수능특강_정유빈의_확률과_통계,01강 원순열,2,791.729465,112.991755
3,38.707664,정유빈,2024_수능특강_정유빈의_확률과_통계,01강 원순열,3,889.447615,15.273605
4,58.073107,정유빈,2024_수능특강_정유빈의_확률과_통계,01강 원순열,4,847.857850,56.863370
...,...,...,...,...,...,...,...
3895,1729.979501,정유빈,2024_수능특강_정유빈의_확률과_통계,40강 도착지는 만점♥ 통계적 추정(Lv. 2),95,801.122860,97.341370
3896,1748.044626,정유빈,2024_수능특강_정유빈의_확률과_통계,40강 도착지는 만점♥ 통계적 추정(Lv. 2),96,835.520175,62.944055
3897,1766.434830,정유빈,2024_수능특강_정유빈의_확률과_통계,40강 도착지는 만점♥ 통계적 추정(Lv. 2),97,812.031000,86.433230
3898,1785.219773,정유빈,2024_수능특강_정유빈의_확률과_통계,40강 도착지는 만점♥ 통계적 추정(Lv. 2),98,850.274950,48.189280


In [ ]:
df1_interval_pitch_deviation.to_csv('./yubin_pitch.csv')

In [ ]:
import numpy as np
import pandas as pd

# 타임스탬프를 각 강의의 시작 지점을 1%, 마지막 지점을 100%로 백분율로 변환하는 함수
def convert_to_percentage_by_lecture(df):
    percentage_list = []

    for lecture_name, group in df.groupby('lecture_name'):
        start_timestamp = group['time_stamp'].min()
        end_timestamp = group['time_stamp'].max()

        group['Scaled_Percent'] = (group['time_stamp'] - start_timestamp) / (end_timestamp - start_timestamp) * 99 + 1
        percentage_list.append(group)

    return pd.concat(percentage_list)

# lecture_name이 같은 값에 해당하는 pitch의 중앙값을 계산하고 deviation 컬럼을 추가하는 함수
def calculate_deviation(df):
    deviation_list = []

    for (teacher, class_name, lecture_name), group in df.groupby(['teacher', 'class_name', 'lecture_name']):
        median_pitch = group['pitch'].median()

        # deviation 계산 및 데이터프레임에 추가
        group['deviation'] = median_pitch - group['pitch']
        deviation_list.append(group)

    return pd.concat(deviation_list)

# 주어진 간격에 따라 중간값의 피치와 deviation 중간값을 계산하는 함수
def calculate_interval_pitch_with_deviation(df, interval):
    df_with_deviation = calculate_deviation(df)
    interval_pitch_deviation = []

    for (teacher, class_name, lecture_name, interval), group in df_with_deviation.groupby(['teacher', 'class_name', 'lecture_name', 'interval']):
        median_pitch = group['pitch'].median()
        median_deviation = group['deviation'].median()

        interval_pitch_deviation.append({
            'time_stamp': group['time_stamp'].iloc[0],  # 첫 번째 타임스탬프 사용
            'teacher': teacher,
            'class_name': class_name,
            'lecture_name': lecture_name,
            'interval': interval,
            'median_pitch': median_pitch,
            'median_deviation': median_deviation
        })

    return pd.DataFrame(interval_pitch_deviation)

# 주어진 데이터프레임을 각 강의의 시작 지점을 1%, 마지막 지점을 100%로 백분율로 변환
df2 = convert_to_percentage_by_lecture(df2)
# lecture_name의 같은 값에 해당되는 강의에서 pitch의 중간값을 구하고 deviation 컬럼을 추가
df2_with_deviation = calculate_deviation(df2)
# interval 컬럼 추가 (간격을 1로 수정)
df2_with_deviation['interval'] = pd.cut(df2_with_deviation['Scaled_Percent'], bins=np.arange(0, 101, 1), labels=False)
# pitch 값과 deviation 값의 중간값을 계산한 데이터프레임 생성
df2_interval_pitch_deviation = calculate_interval_pitch_with_deviation(df2_with_deviation, 1)

df2_interval_pitch_deviation


,time_stamp,teacher,class_name,lecture_name,interval,median_pitch,median_deviation
0,2.461315,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,0,2254.503000,-1393.329400
1,2.554195,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,1,1820.601200,-959.427600
2,9.450522,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,2,1650.150000,-788.976400
3,14.744671,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,3,798.236370,62.937230
4,21.060499,서지나,올림포스_확률과_통계,'확률과 통계' 설득을 위한 수단,4,730.911105,130.262495
...,...,...,...,...,...,...,...
5095,3228.130975,서지나,올림포스_확률과_통계,9강. 이항정리(3),95,895.024275,-1.641215
5096,3261.149751,서지나,올림포스_확률과_통계,9강. 이항정리(3),96,914.148400,-20.765340
5097,3296.699501,서지나,올림포스_확률과_통계,9강. 이항정리(3),97,865.465485,27.917575
5098,3330.298776,서지나,올림포스_확률과_통계,9강. 이항정리(3),98,1350.528600,-457.145540


In [ ]:
df2_interval_pitch_deviation.to_csv('./gina_pitch.csv')

In [ ]:
import numpy as np
import pandas as pd

# 타임스탬프를 각 강의의 시작 지점을 1%, 마지막 지점을 100%로 백분율로 변환하는 함수
def convert_to_percentage_by_lecture(df):
    percentage_list = []

    for lecture_name, group in df.groupby('lecture_name'):
        start_timestamp = group['time_stamp'].min()
        end_timestamp = group['time_stamp'].max()

        group['Scaled_Percent'] = (group['time_stamp'] - start_timestamp) / (end_timestamp - start_timestamp) * 99 + 1
        percentage_list.append(group)

    return pd.concat(percentage_list)

# lecture_name이 같은 값에 해당하는 pitch의 중앙값을 계산하고 deviation 컬럼을 추가하는 함수
def calculate_deviation(df):
    deviation_list = []

    for (teacher, class_name, lecture_name), group in df.groupby(['teacher', 'class_name', 'lecture_name']):
        median_pitch = group['pitch'].median()

        # deviation 계산 및 데이터프레임에 추가
        group['deviation'] = median_pitch - group['pitch']
        deviation_list.append(group)

    return pd.concat(deviation_list)

# 주어진 간격에 따라 중간값의 피치와 deviation 중간값을 계산하는 함수
def calculate_interval_pitch_with_deviation(df, interval):
    df_with_deviation = calculate_deviation(df)
    interval_pitch_deviation = []

    for (teacher, class_name, lecture_name, interval), group in df_with_deviation.groupby(['teacher', 'class_name', 'lecture_name', 'interval']):
        median_pitch = group['pitch'].median()
        median_deviation = group['deviation'].median()

        interval_pitch_deviation.append({
            'teacher': teacher,
            'class_name': class_name,
            'lecture_name': lecture_name,
            'interval': interval,
            'median_pitch': median_pitch,
            'median_deviation': median_deviation
        })

    return pd.DataFrame(interval_pitch_deviation)

# 주어진 데이터프레임을 각 강의의 시작 지점을 1%, 마지막 지점을 100%로 백분율로 변환
df1 = convert_to_percentage_by_lecture(df1)
# lecture_name의 같은 값에 해당되는 강의에서 pitch의 중간값을 구하고 deviation 컬럼을 추가
df1_with_deviation = calculate_deviation(df1)
# interval 컬럼 추가 (간격을 1로 수정)
df1_with_deviation['interval'] = pd.cut(df1_with_deviation['Scaled_Percent'], bins=np.arange(0, 101, 1), labels=False)
# pitch 값과 deviation 값의 중간값을 계산한 데이터프레임 생성
df1_interval_pitch_deviation = calculate_interval_pitch_with_deviation(df1_with_deviation, 1)

df1_interval_pitch_deviation


,teacher,class_name,lecture_name,interval,median_pitch,median_deviation
0,서영란,수능_감(感)_잡기_확률과_통계,01강 [개념] 유형01 원순열과 순열,0,2397.08600,-1200.62320
1,서영란,수능_감(感)_잡기_확률과_통계,01강 [개념] 유형01 원순열과 순열,1,1496.55725,-300.09445
2,서영란,수능_감(感)_잡기_확률과_통계,01강 [개념] 유형01 원순열과 순열,2,1110.91810,85.54470
3,서영란,수능_감(感)_잡기_확률과_통계,01강 [개념] 유형01 원순열과 순열,3,1221.57775,-25.11495
4,서영란,수능_감(感)_잡기_확률과_통계,01강 [개념] 유형01 원순열과 순열,4,1117.74790,78.71490
...,...,...,...,...,...,...
3195,서영란,수능_감(感)_잡기_확률과_통계,32강 [감잡기] 유형16 모평균의 추정,95,1106.32110,33.81405
3196,서영란,수능_감(感)_잡기_확률과_통계,32강 [감잡기] 유형16 모평균의 추정,96,1105.20615,34.92900
3197,서영란,수능_감(感)_잡기_확률과_통계,32강 [감잡기] 유형16 모평균의 추정,97,1009.06555,131.06960
3198,서영란,수능_감(感)_잡기_확률과_통계,32강 [감잡기] 유형16 모평균의 추정,98,1334.29910,-194.16395


### 강사 두 개를 비교하는 정규분포도를 그리려면 아래 코드에서 주석 처리된 코드를 해제하면 됨

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

def cohensd(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    mean1 = np.mean(sample1)
    mean2 = np.mean(sample2)
    std1 = np.std(sample1, ddof=1)
    std2 = np.std(sample2, ddof=1)
    pooled_var = ((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2)
    cohensd = (mean1 - mean2) / np.sqrt(pooled_var)
    return cohensd

# Z-score를 기반으로 이상치 제거하는 함수
def remove_outliers_zscore(df, column, threshold=3):
    z_scores = (df[column] - df[column].mean()) / df[column].std()
    df_filtered = df[np.abs(z_scores) < threshold]
    return df_filtered

median_deviation_median1 = df1.groupby('interval')['median_deviation'].median().reset_index()
median_deviation_median2 = df2.groupby('interval')['median_deviation'].median().reset_index() #

# 각각의 선생님에 대해 이상치 제거
df1 = remove_outliers_zscore(median_deviation_median1, 'median_deviation')
df2 = remove_outliers_zscore(median_deviation_median2, 'median_deviation') #

# 필터링된 데이터에 대한 효과 크기 계산
effect_size_median_filtered = cohensd(df1['median_deviation'], df2['median_deviation']) #

# 일관된 막대 두께를 위한 bin width 지정
bin_width = (df1['median_deviation'].max() - df1['median_deviation'].min()) / 30

# 이상치 제거 후 각각의 선생님에 대한 서로 다른 색상으로 결합된 히스토그램 생성
fig = go.Figure()

fig.add_trace(go.Histogram(x=df1['median_deviation'],
                            nbinsx=30,
                            xbins=dict(start=df1['median_deviation'].min(), end=df1['median_deviation'].max(), size=bin_width),
                            marker_color='blue',
                            opacity=0.5,  # 선생님 1에 대한 투명도 조절
                            name='선생님 1'))

fig.add_trace(go.Histogram(x=df2['median_deviation'],
                            nbinsx=30,
                            xbins=dict(start=df2['median_deviation'].min(), end=df2['median_deviation'].max(), size=bin_width),
                            marker_color='orange',
                            opacity=0.5,  # 선생님 2에 대한 투명도 조절
                            name='선생님 2'))  #

# 레이아웃 및 제목 업데이트
fig.update_layout(title=f'중앙값 편차 분포 (이상치 제거됨)\n효과 크기 (Cohen\'s d): {effect_size_median_filtered:.4f}',
                  xaxis_title='중앙값 편차',
                  yaxis_title='개수',
                  barmode='overlay')  # 히스토그램 겹치기

# 그래프 표시
fig.show()
